# Importar Librerias

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, inspect, text
from sqlalchemy.exc import SQLAlchemyError


### Crear el Engine

In [2]:
engine = create_engine('sqlite:///ejercicio_database.db')

In [3]:
df = pd.read_csv('clientes_banco.csv')
df.head()

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008804,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
2,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio
3,5008808,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
4,5008809,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio


### Guardar en tabla sql

In [4]:
# df.to_sql('clientes_financiera', engine, index=False)

In [4]:
inspector = inspect(engine)
print(inspector.get_table_names())

['clientes_financiera']


In [5]:
pd.read_sql_table('clientes_financiera', con=engine.connect()).head()

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008804,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
2,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio
3,5008808,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
4,5008810,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio


### Actualizacion del Registro ID 6840104

In [ ]:
# actualizar registro ID 6840104
query = "UPDATE clientes_financiera SET Rendimiento_anual=300000.0 WHERE ID_Cliente=6840104"

try:
    with engine.begin() as connection:
        r_set = connection.execute(text(query))
        print("#registros actualizados:", r_set.rowcount)
except SQLAlchemyError as e:
    print(e)

# Verificar el cambio
df_verificado = pd.read_sql_table("clientes_financiera", con=engine)
print(df_verificado[df_verificado['ID_Cliente'] == 6840104])

#registros actualizados: 1
        ID_Cliente  Edad     Grado_estudio Estado_civil  Tamaño_familia  \
438458     6840104    62  Nivel intermedio   Divorciado               1   

       Categoria_de_renta Ocupacion  Años_empleado  Rendimiento_anual  \
438458        Pensionista      Otro              0           300000.0   

        Tiene_carro                  Vivienda  
438458            0  Casa/Departamento propio  


### Verifica si se Actualizo el Registro

In [7]:
# Verificar específicamente el registro
verificar_query = "SELECT * FROM clientes_financiera WHERE ID_Cliente = 6840104"
resultado = pd.read_sql_query(verificar_query, engine)
print("Registro actualizado:")
print(resultado)

Registro actualizado:
   ID_Cliente  Edad     Grado_estudio Estado_civil  Tamaño_familia  \
0     6840104    62  Nivel intermedio   Divorciado               1   

  Categoria_de_renta Ocupacion  Años_empleado  Rendimiento_anual  Tiene_carro  \
0        Pensionista      Otro              0           300000.0            0   

                   Vivienda  
0  Casa/Departamento propio  


### Eliminar Registro ID 5008809

In [7]:
# Borrar el registro ID 5008809
query = "DELETE FROM clientes_financiera WHERE ID_Cliente=5008809"

try:
    with engine.begin() as connection:
        r_set = connection.execute(text(query))
        print("#registro borrado:", r_set.rowcount)
except SQLAlchemyError as e:
    print(e)

# Verificar el cambio
df_borrado= pd.read_sql_table("clientes_financiera", con=engine)
print(df_borrado[df_borrado['ID_Cliente'] == 5008809])

#registro borrado: 0
Empty DataFrame
Columns: [ID_Cliente, Edad, Grado_estudio, Estado_civil, Tamaño_familia, Categoria_de_renta, Ocupacion, Años_empleado, Rendimiento_anual, Tiene_carro, Vivienda]
Index: []


### Creando Nuevo Registro en la BD

In [6]:
query = 'INSERT INTO clientes_financiera (ID_Cliente, Edad, Grado_estudio, Estado_civil, ' \
        'Tamaño_familia, Categoria_de_renta, Ocupacion, Años_empleado, ' \
        'Rendimiento_anual, Tiene_carro, Vivienda) ' \
        'VALUES (6850985, 33, "Doctorado", "Soltero", 1, "Empleado", "TI", ' \
        '2, 290000, 0, "Casa/Departamento propio")'
        
    
try:
    with engine.begin() as connection:
        r_set = connection.execute(text(query))
        print("#registro Nuevo:", r_set.rowcount)
except SQLAlchemyError as e:
    print(e)

# Verificar el cambio
df_nuevo_registro= pd.read_sql_table("clientes_financiera", con=engine)
print(df_nuevo_registro[df_nuevo_registro['ID_Cliente'] == 6850985])     

#registro Nuevo: 1
        ID_Cliente  Edad Grado_estudio Estado_civil  Tamaño_familia  \
438462     6850985    33     Doctorado      Soltero               1   

       Categoria_de_renta Ocupacion  Años_empleado  Rendimiento_anual  \
438462           Empleado        TI              2           290000.0   

        Tiene_carro                  Vivienda  
438462            0  Casa/Departamento propio  


### Verificar los Cambios Realizados

In [7]:
pd.read_sql_table("clientes_financiera", con=engine.connect()).head()

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008804,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
2,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio
3,5008808,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
4,5008810,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
